<a href="https://colab.research.google.com/github/francescopandolfo/TESI_LM32/blob/main/RF%20-%20variazione_relativa%20-%20fontane.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import datetime
from sklearn.utils import shuffle

import warnings
warnings.filterwarnings("ignore")
pathDrive = 'drive/MyDrive/UNICT/_TESI/dati/'

#df = pd.read_csv('sample_data/dati_interpolati_diff_1d-2d_undsamp_7gg.csv', sep=';')
df = pd.read_csv(pathDrive + 'dati_interpolati_diff_1d-2d_8gg - fontane.csv', sep=';')
#df = pd.read_csv('tesi/dati_interpolati_diff_1d_undsamp_7gg_2classi.csv', sep=';')
#df = pd.read_csv('sample_data/dati_interpolati_diff_3d-3d_undsamp_7gg.csv', sep=';')

df['edam_ecne_eplu_diff'] = df['edam_ecne_eplu_diff'].str.replace(',','.').astype(float)#
df['edam_ecor_emcn_diff'] = df['edam_ecor_emcn_diff'].str.replace(',','.').astype(float)
df['edam_emsg_eplu_diff'] = df['edam_emsg_eplu_diff'].str.replace(',','.').astype(float)
df['edam_epdn_ecne_diff'] = df['edam_epdn_ecne_diff'].str.replace(',','.').astype(float)
df['edam_epdn_emcn_diff'] = df['edam_epdn_emcn_diff'].str.replace(',','.').astype(float)
df['eint_ecpn_emgl_diff'] = df['eint_ecpn_emgl_diff'].str.replace(',','.').astype(float)#
df['eint_esln_emgl_diff'] = df['eint_esln_emgl_diff'].str.replace(',','.').astype(float)
df['eint_esln_espc_diff'] = df['eint_esln_espc_diff'].str.replace(',','.').astype(float)
df['emeg_ecpn_emgl_diff'] = df['emeg_ecpn_emgl_diff'].str.replace(',','.').astype(float)
df['emfn_ecor_ecri_diff'] = df['emfn_ecor_ecri_diff'].str.replace(',','.').astype(float)
df['emfn_ecor_emcn_diff'] = df['emfn_ecor_emcn_diff'].str.replace(',','.').astype(float)
df['emfn_emcn_epdn_diff'] = df['emfn_emcn_epdn_diff'].str.replace(',','.').astype(float)
df['epdn_ecne_ecpn_diff'] = df['epdn_ecne_ecpn_diff'].str.replace(',','.').astype(float)
df['epdn_eint_ecpn_diff'] = df['epdn_eint_ecpn_diff'].str.replace(',','.').astype(float)##
df['epdn_eint_espc_diff'] = df['epdn_eint_espc_diff'].str.replace(',','.').astype(float)
df['epdn_emfn_espc_diff'] = df['epdn_emfn_espc_diff'].str.replace(',','.').astype(float)
df['eplu_ecne_ecpn_diff'] = df['eplu_ecne_ecpn_diff'].str.replace(',','.').astype(float)
df['eplu_emeg_ecpn_diff'] = df['eplu_emeg_ecpn_diff'].str.replace(',','.').astype(float)
df['eplu_emeg_emal_diff'] = df['eplu_emeg_emal_diff'].str.replace(',','.').astype(float)
df['eplu_emsg_emal_diff'] = df['eplu_emsg_emal_diff'].str.replace(',','.').astype(float)

df = shuffle(df)
_dftarget = df['evento'].astype(int).values

format = '%d/%m/%Y'
df['date'] = pd.to_datetime(df['day'], format=format)
df = df.set_index('date')
df = df.drop(columns=['day'])
df = df.drop(columns=['evento'])

for i in reversed(range(len(df.columns))):
  if df.dtypes[df.columns[i]] == 'object':
    df = df.drop(df.columns[i], axis=1)

normalized_df = (df - df.mean()) / df.std()
df = normalized_df

#print(df.info())


In [ ]:
from sklearn.model_selection import cross_val_score
import numpy as np
from sklearn.ensemble import RandomForestClassifier

def processa():

  feature_number = 20
  clf = RandomForestClassifier(max_depth=3, random_state=0)

  y = _dftarget
  split_at = 38

  #max_f1_score = 0
  #max_f1_score_who = ""
  max_accuracy_score_test = 0
  max_accuracy_score_test_who = ""

  max_accuracy_score = 0
  max_accuracy_score_who = ""

  for i in range(feature_number):
    for j in range(feature_number):
      for a in range(feature_number):
        for b in range(feature_number):
          for c in range(feature_number):
            for d in range(feature_number):
              #for e in range(feature_number):
                #for f in range(feature_number):
                  #for g in range(feature_number):
                    #for h in range(feature_number):

              if i<j and j<a and a<b and b<c and c<d and i>0:
              #if i==4 and j==7 and a==14:

                X = df.values[:, [i,j,a,b,c,d]]
                who = "area " + str(i) + " - " + str(j) + " - " + str(a) + " - " + str(b) + " - " + str(c) + " - " + str(d)

                X_train = X[:split_at]
                X_test = X[split_at:]

                y_train = y[:split_at]
                y_test = y[split_at:]


                clf.fit(X_train, y_train)

                #Return the mean accuracy on the given test data and labels.
                acc_s = clf.score(X_train, y_train, sample_weight=None)*100
                #acc_s = np.mean(cross_val_score(clf, X_train, y_train, cv=5))*100

                acc_s_test = clf.score(X_test, y_test, sample_weight=None)*100

                #print(clf.predict([[0, 0, 0, 0]]))

                if max_accuracy_score < acc_s+acc_s_test:
                  max_accuracy_score = acc_s+acc_s_test
                  max_accuracy_score_who = who
                  print( who + ";" + "{:.1f}".format(acc_s) + ";" + "{:.1f}".format(acc_s_test) +";" + "{:.1f}".format(acc_s+acc_s_test) )

                else:
                  if max_accuracy_score_test < acc_s_test:
                    max_accuracy_score_test = acc_s_test
                    max_accuracy_score_test_who = who
                    print( who + ";" + "{:.1f}".format(acc_s) + ";" + "{:.1f}".format(acc_s_test) +";" + "{:.1f}".format(acc_s+acc_s_test) )



  #print( max_f1_score_who )
  print( "--------------------- max > " + max_accuracy_score_who + " " + str(max_accuracy_score) )
  print( "--------------------- max test> " + max_accuracy_score_test_who + " " + str(max_accuracy_score_test) )
  #print( conf_mat )

processa()

area 1 - 2 - 3 - 4 - 5 - 6;97.4;40.0;137.4
area 1 - 2 - 3 - 4 - 5 - 7;89.5;46.7;136.1
area 1 - 2 - 3 - 4 - 5 - 12;89.5;60.0;149.5
area 1 - 2 - 3 - 4 - 5 - 14;89.5;53.3;142.8
area 1 - 2 - 3 - 4 - 6 - 14;97.4;53.3;150.7
area 1 - 2 - 3 - 4 - 6 - 17;100.0;60.0;160.0
area 1 - 2 - 3 - 4 - 9 - 15;92.1;60.0;152.1
area 1 - 2 - 3 - 4 - 13 - 17;97.4;66.7;164.0
area 1 - 2 - 3 - 5 - 6 - 17;94.7;66.7;161.4
area 1 - 2 - 6 - 10 - 13 - 15;100.0;66.7;166.7
